In [79]:
#!pip install transformers
#!pip install jinja2 --upgrade

In [80]:
#from huggingface_hub import notebook_login
#notebook_login()

In [81]:
#!pip install pynif

#Steps:

    ## Define gold standard MD:

        ### Input:  dataset

        ### Output: mentions

    * Generate candidates for each (DBpediaLookupFinder)

    * Query DBp

    * Generate prompt

In [82]:
# Load mention data
from pynif import NIFCollection

nif_data_path = "C:\\Users\\wf7467\\Desktop\\Evaluation Datasets\\Datasets\\entity_linking\\conll_aida-yago2-dataset\\AIDA-YAGO2-dataset.tsv_nif"

nif_data = ""

with open(nif_data_path, 'r', encoding="utf-8") as f:
    nif_data = f.read()

parsed_collection = NIFCollection.loads(nif_data, format='turtle')

In [83]:
for context in parsed_collection.contexts:
    # See what can be called on context
    # print(dir(context))

    # input sentence (1 document = 1 sentence)
    document = context.mention
    print(f"{context.mention}, {context.beginIndex}, {context.endIndex}")
    for phrase in context.phrases:
        #print(phrase)
        #print(dir(phrase))
        print(f"{phrase.mention}, {phrase.beginIndex}, {phrase.endIndex}, {phrase.generated_uri}, {phrase.taClassRef}, {phrase.taIdentRef}")
    break

Stork H1 results breakdown per sector .  AMSTERDAM 1996-08-28  First 24 weeks 1996  ( millions of guilders unless otherwise stated )  Industrial systems and components  - Turnover 756 vs 829  - Operating profit 46 vs 48  - New orders received 876 vs 933  - Order book ( billions ) 1.07 vs 0.98  Industrial services  - Turnover 657 vs 700  - Operating profit 9 vs 3  - New orders received ( billions ) 1.00 vs 1.09  - Order book ( billions ) 2.37 vs 2.01  NOTE - Order book figures refer to value of orders on books at end of period .  -- Amsterdam newsroom +31 20 504 5000 , Fax +31 20 504 5040 , 0, 595
AMSTERDAM, 41, 50, https://aifb.kit.edu/conll/768#offset_41_50, None, http://en.wikipedia.org/wiki/Amsterdam
Amsterdam, 538, 547, https://aifb.kit.edu/conll/768#offset_538_547, None, http://en.wikipedia.org/wiki/Amsterdam


In [84]:
#!pip install requests

In [85]:
import requests

endpoint_url = "https://lookup.dbpedia.org/api/search?format=JSON&query="

response = requests.get(endpoint_url+"Steve")
print(response.status_code)

200


In [86]:
import json
json_candidates = json.loads(response.text)

In [87]:
# Playing around with the JSON
i = 10

json_documents = json_candidates['docs']
document = json_documents[i]
print(document.keys())
uri = document['resource']
small_desc = document['comment']
types = document['typeName']
label = document['label']
#json_candidates['docs'][i].keys()
f"{types, label, small_desc}"

dict_keys(['score', 'refCount', 'resource', 'redirectlabel', 'typeName', 'comment', 'label', 'id', 'type', 'category'])


"(['MusicalArtist', 'Person', 'Artist', 'Agent'], ['<B>Steve</B> Roach (musician)'], ['<B>Steve</B> Roach (born February 16, 1955) is an American composer and performer of ambient and'])"

In [88]:
def transform_dbp_json_cands_to_dict(json_documents={}):
    candidate_dict = {}
    candidate_dict['types'] = []
    candidate_dict['label'] = []
    candidate_dict['desc'] = []
    candidate_dict['uri'] = []
    candidate_dict['score'] = []
    candidate_dict['refCount'] = []
    
    for i in range(len(json_documents)):
        document = json_documents[i]
        try:
            # Retrieve information
            # Description
            small_desc = document.get('comment', "")
            # Types for this candidate
            types = document.get('typeName', [])
            if types is None:
                #print(f"No types for: {document.keys()}, {document['label']}")
                types = ""
                
            # The label / name for this candidate
            label = document.get('label', [])
            
            # URI for this candidate
            uri = document.get('resource', [])

            # Score (Optional)
            score = document.get('score', [])

            # Reference Count (Optional)
            refCount = document.get('refCount', [])

            # Populate information
            candidate_dict['types'].append(types)
            candidate_dict['label'].extend(label)
            candidate_dict['desc'].extend(small_desc)
            candidate_dict['uri'].extend(uri)
            candidate_dict['score'].extend(score)
            candidate_dict['refCount'].extend(refCount)

            
        except:
            print(f"ERROR: {document.keys()}")
            raise
    return candidate_dict['label'], candidate_dict['desc'], candidate_dict['types'], candidate_dict['uri'], candidate_dict['score'], candidate_dict['refCount']


#labels, desc, types, uris, score, refCount = transform_dbp_json_cands_to_dict(json_documents)

In [89]:
import json

def generate_candidates(mention):
    endpoint_url = "https://lookup.dbpedia.org/api/search?format=JSON&query="

    response = requests.get(endpoint_url+str(mention))
    if response.status_code != 200:
        raise ValueError(f"Could not retrieve candidates for mention[{mention}], code: {response.status_code}")
    # Response to JSON
    json_candidates = json.loads(response.text)
    
    # Extract the candidate "documents" from the response JSON
    json_candidate_documents = json_candidates['docs']
    
    # Transform DBpedia JSON candidates to data structure we want to use for LLM purposes
    labels, desc, types, uris, score, refCount = transform_dbp_json_cands_to_dict(json_documents=json_candidate_documents)
    return [labels, desc, types, uris, score, refCount]

#lst_candidate_info = generate_candidates("Steve")

In [90]:
#Let's see some type names
#set([x for y in candidate_dict['types'] for x in y])

In [91]:
import json
from os import listdir
from os.path import isfile, join

def save_candidate_result(cand_dictionary={}, \
                          result_directory="C:/Users/wf7467/Desktop/GitHub/KIT/agnostic-disambiguation/data/candidates/CoNLL_AIDA-YAGO2-dataset.nif/",\
                          file_counter="NA"):
    with open(result_directory+str(file_counter)+'.json', 'w', encoding='utf-8') as f:
        json.dump(cand_dictionary, f, ensure_ascii=False, indent=4)
        
def load_candidate_results(
                          result_directory="C:/Users/wf7467/Desktop/GitHub/KIT/agnostic-disambiguation/data/candidates/CoNLL_AIDA-YAGO2-dataset.nif/"):
    all_candidate_results = {}
    for path in [result_directory+f for f in listdir(result_directory) if isfile(join(result_directory, f))]:
        with open(path, encoding='utf-8') as json_file:
            in_json = json.load(json_file)
            
            all_candidate_results[path] = in_json
    return all_candidate_results

In [92]:
#!pip install tqdm

In [93]:
# Load mention data
from pynif import NIFCollection

def load_nif_dataset(nif_data_path = "C:\\Users\\wf7467\\Desktop\\Evaluation Datasets\\Datasets\\entity_linking\\conll_aida-yago2-dataset\\AIDA-YAGO2-dataset.tsv_nif"):

    nif_data = ""

    with open(nif_data_path, 'r', encoding="utf-8") as f:
        nif_data = f.read()

    parsed_collection = NIFCollection.loads(nif_data, format='turtle')

parsed_collection = load_nif_dataset(nif_data_path = "C:\\Users\\wf7467\\Desktop\\Evaluation Datasets\\Datasets\\entity_linking\\conll_aida-yago2-dataset\\AIDA-YAGO2-dataset.tsv_nif")

In [94]:
# Generate dictionary of results that will be persisted for generating prompts from

#from tqdm import tqdm
from tqdm import tqdm_notebook

import json
from os import listdir
from os.path import isfile, join

def compute_save_candidates_for_mentions(result_directory="C:/Users/wf7467/Desktop/GitHub/KIT/agnostic-disambiguation/data/candidates/CoNLL_AIDA-YAGO2-dataset.nif/"):
# Check which are done in directory so we don't redo them

    # to find what is where & avoid redoing
    mapping_path_document = {}
    for path in [result_directory+f for f in listdir(result_directory) if isfile(join(result_directory, f))]:
        with open(path, encoding='utf-8') as json_file:
            in_json = json.load(json_file)
        key = list(dict(in_json).keys())[0]
        mapping_path_document[key] = path

    file_counter = len(mapping_path_document)
    # To see how many were skipped
    skip_counter = 0
    
    for context in tqdm_notebook(parsed_collection.contexts, position=0):
        mapping_document_candidate_info = {}

        # See what can be called on context
        # print(dir(context))

        # document/context.mention = input sentence (1 document = 1 sentence)
        document = context.mention
        # If it's done already, continue to next
        if document in mapping_path_document.keys():
            skip_counter += 1
            continue

        # Document string will be used as key for all candidate information
        key = document

        mentions = []

        doc_labels = []
        doc_desc = []
        doc_types = []
        doc_candidate_uris = []
        doc_scores = []
        doc_refCount = []
        doc_true_uris = []
        doc_begin_index = []
        doc_end_index = []


        #print(f"{context.mention}, {context.beginIndex}, {context.endIndex}")
        # 1 phrase = 1 mention
        for phrase in tqdm_notebook(context.phrases, position=1):
            #print(phrase)
            #print(dir(phrase))
            #print(f"{phrase.mention}, {phrase.beginIndex}, {phrase.endIndex}, {phrase.generated_uri}, {phrase.taClassRef}, {phrase.taIdentRef}")
            lst_candidate_info = generate_candidates(phrase.mention)
            labels = lst_candidate_info[0]
            desc = lst_candidate_info[1]
            types = lst_candidate_info[2]
            candidate_uris = lst_candidate_info[3]
            scores = lst_candidate_info[4]
            refCount = lst_candidate_info[5]
            # Single URI
            true_uri = phrase.taIdentRef

            # Add data for persistence
            # always single entry per mention --> extend
            # multiple entries per mention    --> append
            mentions.append(phrase.mention)
            doc_labels.append(labels)
            doc_desc.append(desc)
            doc_types.append(types)
            doc_candidate_uris.append(candidate_uris)
            doc_scores.append(scores)
            doc_refCount.append(refCount)
            # Single URI
            doc_true_uris.append(true_uri)
            doc_begin_index.append(phrase.beginIndex)
            doc_end_index.append(phrase.endIndex)


        # Once all phrases aka. mentions for a sentence are done, store the sentence information
        mapping_document_candidate_info[key] = {
            'mentions' : mentions,#
            'begin_index' : doc_begin_index,#
            'end_index' : doc_end_index,#
            'labels' : doc_labels,#
            'desc' : doc_desc,#
            'types' : doc_types,#
            'candidate_uris' : doc_candidate_uris,#
            'scores' : doc_scores,#
            'refCount' : doc_refCount,#
            'true_uris' : doc_true_uris,#
        }

        # Save results since sentence has been processed

        save_candidate_result(cand_dictionary=mapping_document_candidate_info, file_counter=file_counter)
        file_counter+=1
        #if file_counter >= 2:
        #    break

    print(f"Finished processing {file_counter} documents, skipped {skip_counter}.")

In [95]:
all_candidate_results = load_candidate_results(result_directory="C:/Users/wf7467/Desktop/GitHub/KIT/agnostic-disambiguation/data/candidates/CoNLL_AIDA-YAGO2-dataset.nif/")
print(f"Number of entries: {len(all_candidate_results)}")

Number of entries: 1393


In [96]:
#get some random key to get one entry
key = (list(all_candidate_results.keys()))[10]
#print(all_candidate_results[list(all_candidate_results.keys())[10]])
print(key)
#key = next(iter(all_candidate_results))
candidates_entry = all_candidate_results[key]
# Only has one key
in_text = next(iter( candidates_entry ))
json_candidates = candidates_entry[in_text]
json_candidates.keys()

C:/Users/wf7467/Desktop/GitHub/KIT/agnostic-disambiguation/data/candidates/CoNLL_AIDA-YAGO2-dataset.nif/1006.json


dict_keys(['mentions', 'begin_index', 'end_index', 'labels', 'desc', 'types', 'candidate_uris', 'scores', 'refCount', 'true_uris'])

In [97]:
# Go through all entries, grab the URIs ('candidate_uris' and 'true_uris')
# Make a set out of them
# Query SPARQL for their descriptions via DBpedia
# Save them all in a dictionary fashion: URI --> text description
# Save to file
# Load it properly




In [98]:
# Step 1: For which URIs do we need additional info (aka. descriptions)?

wanted_keys = ['candidate_uris', 'true_uris']
lst_len_counter = 0
uris_for_descriptions = set()

for in_file in all_candidate_results:
    entry = all_candidate_results[in_file]
    #print(in_file, entry.keys())

    for document in entry:
        document_candidate_json = entry[document]
        for key in document_candidate_json:
            if key in wanted_keys:
                uris_of_uris = document_candidate_json[key]
                for uris in uris_of_uris:
                    lst_len_counter = lst_len_counter + len(uris)
                    uris_for_descriptions.update(uris)
                #print(len(uris_for_descriptions))
                #print(lst_len_counter)
            #break
        #break
    #break

print(lst_len_counter)
print(len(uris_for_descriptions))
print(next(iter(uris_for_descriptions)))
uris_for_descriptions = list(uris_for_descriptions)


3944986
361239
http://dbpedia.org/resource/Paul_Whelan


In [99]:
# Step 2: Query SPARQL for their descriptions via DBpedia
# https://sparqlwrapper.readthedocs.io/en/latest/main.html
#!pip install sparqlwrapper
from SPARQLWrapper import SPARQLWrapper, JSON, POST

def build_get_description(dbpedia_subject_uri=None):
    if dbpedia_subject_uri is None:
        raise ValueError("You need to input a subject URI/IRI")
    return """
        {
            ?s dbo:abstract ?desc .
            FILTER(?s = <"""+dbpedia_subject_uri+""">) . 
            FILTER(LANG(?desc) = "en") .
        }
"""

def build_union_query_get_description(dbpedia_subject_uris=[]):
    if dbpedia_subject_uris is None:
        raise ValueError("You need to input a subject URI/IRI")
    union_query_start = """
    SELECT DISTINCT ?s ?desc
    WHERE
    {
    """
    center_query = ""
    union_query_end = "}"

    # Take the first element separately... makes it a lot easier for the logic
    center_query += build_get_description(str(dbpedia_subject_uris[0]))

    for idx in range(1, len(dbpedia_subject_uris)):
        uri = str(dbpedia_subject_uris[idx])
        center_query += """
        UNION
        """
        center_query += build_get_description(uri)
    return union_query_start + center_query + union_query_end


def query_descriptions_multiple_uris(uris_for_descriptions):
    '''Note that DBpedia's SPARQL endpoint can only handle 10k output - so definitely make it smaller than that.'''
    query = build_union_query_get_description(uris_for_descriptions)

    sparql = SPARQLWrapper(endpoint="https://dbpedia.org/sparql", returnFormat=JSON)
    sparql.setMethod(POST)

    # test_query = """
    #     SELECT DISTINCT ?s ?desc
    #     WHERE
    #     {
    #         { 
    #             ?s dbo:abstract ?desc .
    #             FILTER(?s = <http://dbpedia.org/resource/Steve_Jobs>) . 
    #             FILTER(LANG(?desc) = "en") .
    #         }
    #         UNION
    #         { 
    #             ?s dbo:abstract ?desc .
    #             FILTER(?s = <http://dbpedia.org/resource/Steve_Wozniak>) . 
    #             FILTER(LANG(?desc) = "en") .
    #         }
    #     }
    #     """

    sparql.setQuery(query)

    dict_uri_desc = {}

    try:
        ret = sparql.queryAndConvert()


        for r in ret["results"]["bindings"]:
            uri = r['s']['value']
            desc = r['desc']['value']
            dict_uri_desc[uri] = desc
    except Exception as e:
        print(e)

    return dict_uri_desc



In [100]:
import json
from os import listdir
from os.path import isfile, join

def save_cand_desc_result(cand_desc_dictionary={}, \
                          result_directory="C:/Users/wf7467/Desktop/GitHub/KIT/agnostic-disambiguation/data/candidate_descriptions/CoNLL_AIDA-YAGO2-dataset.nif/",\
                          file_counter="NA"):
    with open(result_directory+str(file_counter)+'.json', 'w', encoding='utf-8') as f:
        json.dump(cand_desc_dictionary, f, ensure_ascii=False, indent=4)
        
def load_cand_desc_results(
                          result_directory="C:/Users/wf7467/Desktop/GitHub/KIT/agnostic-disambiguation/data/candidate_descriptions/CoNLL_AIDA-YAGO2-dataset.nif/"):
    all_cand_desc_results = {}
    for path in [result_directory+f for f in listdir(result_directory) if isfile(join(result_directory, f))]:
        with open(path, encoding='utf-8') as json_file:
            in_json = json.load(json_file)
            
            all_cand_desc_results[path] = in_json
    return all_cand_desc_results

In [101]:
# Step of 100 would hopefully work
step = 100
file_counter_desc = 0
for i in range(0, len(uris_for_descriptions), step):
    print("Working on range[%d, %d]" % (i, i+step))
    dict_uri_desc = query_descriptions_multiple_uris(uris_for_descriptions[i:i+step])
    save_cand_desc_result(cand_desc_dictionary=dict_uri_desc, file_counter=i)
    file_counter_desc += 1


Working on range[0, 100]
Working on range[100, 200]
Working on range[200, 300]
Working on range[300, 400]
Working on range[400, 500]
Working on range[500, 600]
Working on range[600, 700]
Working on range[700, 800]
Working on range[800, 900]
Working on range[900, 1000]
Working on range[1000, 1100]
Working on range[1100, 1200]
Working on range[1200, 1300]
Working on range[1300, 1400]
Working on range[1400, 1500]
Working on range[1500, 1600]
Working on range[1600, 1700]
Working on range[1700, 1800]
Working on range[1800, 1900]
Working on range[1900, 2000]
Working on range[2000, 2100]
Working on range[2100, 2200]
Working on range[2200, 2300]
Working on range[2300, 2400]
Working on range[2400, 2500]
Working on range[2500, 2600]
Working on range[2600, 2700]
Working on range[2700, 2800]
Working on range[2800, 2900]
Working on range[2900, 3000]
Working on range[3000, 3100]
Working on range[3100, 3200]
Working on range[3200, 3300]
Working on range[3300, 3400]
Working on range[3400, 3500]
Workin

KeyboardInterrupt: 

In [123]:
# Now double check whether all of the URIs we have also actually have an associated description
# If not, ... crawl the missing ones again.
def check_for_missing_uri_descriptions_and_complete(uris_for_descriptions=[], step=100, prefix=""):
    all_cand_desc_results = load_cand_desc_results()

    # uris_for_descriptions --> all URIs we want to crawl
    # cands_desc
    uris_for_descriptions_crawled = set()
    for filename in all_cand_desc_results:
        cands_desc = all_cand_desc_results[filename]
        #print(list(cands_desc.keys())[0])
        uris_for_descriptions_crawled.update(set(cands_desc.keys()))

    set_missing_uris = set(uris_for_descriptions) - set(uris_for_descriptions_crawled)
    lst_missing_uris = list(set_missing_uris)
    #print(lst_missing_uris[0:50])

    print("%d missing URIs (crawled: %d, total wanted: %d)." % (len(set_missing_uris), len(set(uris_for_descriptions_crawled)), len(set(uris_for_descriptions))))

    for i in range(0, len(lst_missing_uris), step):
        print("Working on missing range[%d, %d]" % (i, i+step))
        dict_uri_desc = query_descriptions_multiple_uris(lst_missing_uris[i:i+step])
        save_cand_desc_result(cand_desc_dictionary=dict_uri_desc, file_counter=prefix+str(i))


In [122]:
# Checking 3 times for missing URIs... if they continue missing, it means they don't have an abstract. Most are due to being disambiguation pages
# Example of missing URIs:
# ['http://dbpedia.org/resource/Paul_Whelan', 'http://dbpedia.org/resource/Rovereto_(disambiguation)', 'http://dbpedia.org/resource/Irwin', 'http://dbpedia.org/resource/CEFCU', 'http://dbpedia.org/resource/Angolan_swallow', 'http://dbpedia.org/resource/Mountain_View,_New_Mexico', 'http://dbpedia.org/resource/Siege_of_Damascus', 'http://dbpedia.org/resource/Jerry_Simmons', 'http://dbpedia.org/resource/Frank_Richter', 'http://dbpedia.org/resource/CGU', 'http://dbpedia.org/resource/El_Gran_campeón', 'http://dbpedia.org/resource/Tsentaroy', 'http://dbpedia.org/resource/DoÃ±a_Ana_County,_New_Mexico', 'http://dbpedia.org/resource/Bethlehem_High_School', 'http://dbpedia.org/resource/List_of_unnumbered_minor_planets:_2003_A–R', 'http://dbpedia.org/resource/SPAP', 'http://dbpedia.org/resource/Charleston_Civic_Center', 'http://dbpedia.org/resource/ASAC', 'http://dbpedia.org/resource/Robert_Irwin', 'http://dbpedia.org/resource/Maarit_LepomÃ¤ki', 'http://dbpedia.org/resource/PloieÈ™ti', 'http://dbpedia.org/resource/Quattro_Canti,_Catania', 'http://dbpedia.org/resource/Peruvian_Brazilians', "http://dbpedia.org/resource/Church_of_Sant'Angelo,_Perugia", 'http://dbpedia.org/resource/Yelena_Vyalbe', 'http://dbpedia.org/resource/Saint-cyriens', 'http://dbpedia.org/resource/1994_Milan_Indoor_–_Singles', 'http://dbpedia.org/resource/Tuqtu', 'http://dbpedia.org/resource/Military_of_Mauritius', 'http://dbpedia.org/resource/Alistar_Fredericks', "http://dbpedia.org/resource/2002_Kroger_St._Jude_International_–_Women's_Doubles", 'http://dbpedia.org/resource/Wayne_Simmons', 'http://dbpedia.org/resource/Lindsay_(name)', 'http://dbpedia.org/resource/List_of_freshman_class_members_of_the_111th_United_States_Congress', 'http://dbpedia.org/resource/Neohumanism', 'http://dbpedia.org/resource/Kenan_IÅŸÄ±k', 'http://dbpedia.org/resource/Belfast_Duncairn', 'http://dbpedia.org/resource/Hum_Sab_Chor_Hain', 'http://dbpedia.org/resource/Flamengo_(disambiguation)', 'http://dbpedia.org/resource/Hà_Tĩnh_Province', 'http://dbpedia.org/resource/List_of_KDE_applications', 'http://dbpedia.org/resource/Sandusky_(surname)', 'http://dbpedia.org/resource/Kishibe', 'http://dbpedia.org/resource/Toyota_concept_vehicles_(1970–1979)', 'http://dbpedia.org/resource/FranÃ§ois_PÃ©ron', 'http://dbpedia.org/resource/WGCL-TV', "http://dbpedia.org/resource/2017_Ricoh_Open_–_Women's_Doubles", 'http://dbpedia.org/resource/KB_Financial_Group_Inc', 'http://dbpedia.org/resource/Arjun_Singh_(politician,_born_1962)', 'http://dbpedia.org/resource/103rd_Infantry_Division_Piacenza']

check_for_missing_uri_descriptions_and_complete(uris_for_descriptions=uris_for_descriptions, step=100, prefix="missing_a_")
check_for_missing_uri_descriptions_and_complete(uris_for_descriptions=uris_for_descriptions, step=100, prefix="missing_b_")
check_for_missing_uri_descriptions_and_complete(uris_for_descriptions=uris_for_descriptions, step=100, prefix="missing_c_")

['http://dbpedia.org/resource/Paul_Whelan', 'http://dbpedia.org/resource/Rovereto_(disambiguation)', 'http://dbpedia.org/resource/Irwin', 'http://dbpedia.org/resource/CEFCU', 'http://dbpedia.org/resource/Angolan_swallow', 'http://dbpedia.org/resource/Mountain_View,_New_Mexico', 'http://dbpedia.org/resource/Siege_of_Damascus', 'http://dbpedia.org/resource/Jerry_Simmons', 'http://dbpedia.org/resource/Frank_Richter', 'http://dbpedia.org/resource/CGU', 'http://dbpedia.org/resource/El_Gran_campeón', 'http://dbpedia.org/resource/Tsentaroy', 'http://dbpedia.org/resource/DoÃ±a_Ana_County,_New_Mexico', 'http://dbpedia.org/resource/Bethlehem_High_School', 'http://dbpedia.org/resource/List_of_unnumbered_minor_planets:_2003_A–R', 'http://dbpedia.org/resource/SPAP', 'http://dbpedia.org/resource/Charleston_Civic_Center', 'http://dbpedia.org/resource/ASAC', 'http://dbpedia.org/resource/Robert_Irwin', 'http://dbpedia.org/resource/Maarit_LepomÃ¤ki', 'http://dbpedia.org/resource/PloieÈ™ti', 'http://dbpe

In [124]:
# Now that we have crawled as much as we could, we should remove the unfindable ones from our database
def populate_candidate_description_dict():
    all_cand_desc_results = load_cand_desc_results()

    uris_for_descriptions_crawled = {}
    for filename in all_cand_desc_results:
        cands_desc = all_cand_desc_results[filename]
        #print(list(cands_desc.keys())[0])
        for key in cands_desc:
            uris_for_descriptions_crawled[key] = cands_desc[key]
    return uris_for_descriptions_crawled

dict_candidate_description = populate_candidate_description_dict()

In [127]:
# Sanity check / testing by outputting a random candidate and its description
cand_uri = list(dict_candidate_description.keys())[0]
description = dict_candidate_description[cand_uri]

print(cand_uri)
print(description)

http://dbpedia.org/resource/New_Amsterdam,_Guyana
New Amsterdam (Dutch: Nieuw Amsterdam) is the regional capital of East Berbice-Corentyne, Guyana and one of the country's largest towns. It is 100 kilometres (62 mi) from the capital, Georgetown and located on the eastern bank of the Berbice River, 6 km (4 mi) upriver from its mouth at the Atlantic Ocean, and immediately south of the Canje River. New Amsterdam's population is 17,329 inhabitants as of 2012.


Now data has been crawled appropriately. Next steps are to use an LLM with the description

TODOs for evaluation:</br>
* Grab descriptions for all URIs via SPARQL query
* sameAs links</br>
* is dbo:wikiPageRedirects of</br>
        * Example: http://en.wikipedia.org/wiki/People's_Republic_of_China vs. http://dbpedia.org/resource/China</br>
        * Relevant file: C:/Users/wf7467/Desktop/GitHub/KIT/agnostic-disambiguation/data/candidates/CoNLL_AIDA-YAGO2-dataset.nif/1.json</br>
    * dbr:People's_Republic_Of_China</br>
